In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import AutoAugment, ColorJitter, Compose

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset_train(Dataset):
    def __init__(self, root_dir, transform=None):
        self.classes = sorted(os.listdir(root_dir))  # get a list of subdirectory names
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}  # map class names to indices
        self.samples = []
        for target_class in self.classes:
            class_dir = os.path.join(root_dir, target_class)
            if not os.path.isdir(class_dir):
                continue
            for root, _, fnames in sorted(os.walk(class_dir)):
                for fname in sorted(fnames):
                    if fname.endswith('.jpg') or fname.endswith('.png'):
                        path = os.path.join(root, fname)
                        item = (path, self.class_to_idx[target_class])
                        self.samples.append(item)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # resize the image
            transforms.ToTensor(),  # convert PIL image to a PyTorch tensor
            
            
        #])
            
            
            #MobileNet with Landmarks
            #transforms.Normalize(mean=[0.4514, 0.3094, 0.2865], 
            #              std=[0.1353, 0.1066, 0.0966]),])
            
            #MobileNet without Landmarks
            transforms.Normalize(mean=[0.4429, 0.3043, 0.2806], 
                          std=[0.1187, 0.0874, 0.0728]),])
        
        if transform is not None:
            self.transform = transforms.Compose([self.transform, transform])
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        with open(path, 'rb') as f:
            sample = Image.open(f).convert('RGB')
        sample = self.transform(sample)
        return sample, target
    
    def __len__(self):
        return len(self.samples)


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset_valid(Dataset):
    def __init__(self, root_dir, transform=None):
        self.classes = sorted(os.listdir(root_dir))  # get a list of subdirectory names
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}  # map class names to indices
        self.samples = []
        for target_class in self.classes:
            class_dir = os.path.join(root_dir, target_class)
            if not os.path.isdir(class_dir):
                continue
            for root, _, fnames in sorted(os.walk(class_dir)):
                for fname in sorted(fnames):
                    if fname.endswith('.jpg') or fname.endswith('.png'):
                        path = os.path.join(root, fname)
                        item = (path, self.class_to_idx[target_class])
                        self.samples.append(item)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # resize the image
            transforms.ToTensor(),  # convert PIL image to a PyTorch tensor
            transforms.Normalize(mean=[0.4429, 0.3043, 0.2806], 
                          std=[0.1187, 0.0874, 0.0728]),])
        
        if transform is not None:
            self.transform = transforms.Compose([self.transform, transform])
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        with open(path, 'rb') as f:
            sample = Image.open(f).convert('RGB')
        sample = self.transform(sample)
        return sample, target
    
    def __len__(self):
        return len(self.samples)


In [ ]:
train_dataset = CustomDataset_train(root_dir='C:/Users/Bruss/Desktop/Speciale/data/used_data_5/training_data')
validation_data = CustomDataset_valid(root_dir='C:/Users/Bruss/Desktop/Speciale/data/used_data_5/testing_data')

In [ ]:
train_loader = DataLoader(train_dataset, batch_size =26, shuffle = True)

val_loader = DataLoader(validation_data, batch_size = 26, shuffle = False)

In [ ]:
# Use a loop to calculate the mean and standard deviation of your training data
def calculate_mean_std(train_loader): 
    mean = 0.
    std = 0.
    for images, _ in train_loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    # Calculate the mean and standard deviation of your training data
    mean /= len(train_loader.dataset)
    std /= len(train_loader.dataset)
    print(f'Mean: {mean}')
    print(f'Standard deviation: {std}')
    return mean, std

In [ ]:
#calculate_mean_std(train_loader)

In [ ]:
import torch
import torch.nn as nn
import torchvision

class MobileNetCNN(nn.Module):
    def __init__(self, num_classes, hidden_size=256, num_layers=2, dropout=0.25):
        super().__init__()
        self.mobilenet = torchvision.models.mobilenet_v2(pretrained=True).features
        for param in self.mobilenet.parameters():
            param.requires_grad = False
            
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.lstm = nn.LSTM(input_size=1280, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size * 2, 512)
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.mobilenet(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = x.unsqueeze(1)
        x = x.expand(-1, self.lstm.num_layers * 2, -1)
        x, _ = self.lstm(x)
        x = self.dropout1(torch.cat((x[:, -1, :self.lstm.hidden_size], x[:, 0, self.lstm.hidden_size:]), dim=1))
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from torchvision.transforms import transforms
from torchsummary import summary

class EfficientNetCNN(nn.Module):
    def __init__(self, num_classes, hidden_size=256, num_layers=2, dropout=0.25):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        #for param in self.efficientnet.parameters():
        #    param.requires_grad = False
        self.lstm = nn.LSTM(input_size=1280, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size * 2, 256)
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.efficientnet.extract_features(x)
        # Flatten the feature maps and add a batch dimension
        x = x.flatten(start_dim=2).permute(0, 2, 1)
        # Pass the flattened feature maps through the bidirectional LSTM layer
        x, _ = self.lstm(x)
        # Concatenate the outputs of the forward and backward directions and pass through the fully connected layers with dropout
        x = self.dropout1(torch.cat((x[:, -1, :self.lstm.hidden_size], x[:, 0, self.lstm.hidden_size:]), dim=1))
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from torch.utils.mobile_optimizer import optimize_for_mobile
from torchvision.transforms import transforms
import matplotlib.pyplot as plt


# set seed for reproducibility
torch.manual_seed(1234)

# rest of the code for training the model

# Save the model to a directory
import os
directory = 'C:/Users/Bruss/Desktop/Speciale/models'
if not os.path.exists(directory):
    os.makedirs(directory)

# Define training parameters
batch_size = 26
#9 epochs

#EFFECIENTNET WITH LANDMARKS: 7 EPOCHS - LOWEST LOSS SECOND HIGHEST VAL lr = 0.0001 /TRAINED ON FOR 20
#EFFECIENTNET WITHOUT LANDMARKS: 10 EPOCHS - LOWEST LOSS SECOND HIGHEST VAL lr = 0.0005 /TRAIN ON FOR 20 #EPOCH 5 BEST YESTERDAY
num_epochs = 20
learning_rate = 0.0001


# Instantiate the model and optimizer
# Instantiate the model and optimizer
model = MobileNetCNN(num_classes=5)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


# Training loop
train_loss_list, val_loss_list = [], []
train_acc_list, val_acc_list = [], []
val_auc_list = []

train_loss_list_graph, val_loss_list_graph = [], []
train_acc_list_graph, val_acc_list_graph = [], []

for epoch in range(num_epochs):
    # Train the model
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        _, predicted = torch.max(outputs.data, 1)
        train_acc_list.append((predicted == labels).sum().item())

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_acc = 0.0
        val_auc = 0.0
        all_labels = []
        all_predictions = []
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss_list.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            val_acc_list.append((predicted == labels).sum().item())
            all_labels.extend(labels.tolist())
            all_predictions.extend(predicted.tolist())



    # Print the results for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss_list[-1]:.4f}, "
          f"Train Acc: {train_acc_list[-1]/len(train_loader.dataset)*10000:.2f}%, "
          f"Val Loss: {val_loss_list[-1]:.4f}, "
          f"Val Acc: {val_acc_list[-1]/len(val_loader.dataset)*100:.2f}%, ")
    train_loss_list_graph.append(train_loss_list[-1])
    val_loss_list_graph.append(val_loss_list[-1])
    train_acc_list_graph.append(train_acc_list[-1]/len(train_loader.dataset)*10000)
    val_acc_list_graph.append(val_acc_list[-1]/len(val_loader.dataset)*100)
    
    #Save model
    #torch.save(model, os.path.join(directory,f'current_model_MobileNet_without_landmarks_{epoch}.pt'))

    
    



In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
# Display the confusion matrix
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(conf_matrix)
ax.set_xticks(range(len(val_loader.dataset.classes)))
ax.set_yticks(range(len(val_loader.dataset.classes)))
ax.set_xticklabels(val_loader.dataset.classes, rotation=90)
ax.set_yticklabels(val_loader.dataset.classes)
for i in range(len(val_loader.dataset.classes)):
    for j in range(len(val_loader.dataset.classes)):
        ax.text(j, i, conf_matrix[i, j], ha="center", va="center", color="white")
ax.set_title("Confusion Matrix")
plt.show()

In [ ]:
# Set up the figure for plotting
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].set_title("Loss")
axs[1].set_title("Accuracy")
# Update the plots
axs[0].plot(train_loss_list_graph, label="Training Loss")
axs[0].plot(val_loss_list_graph, label="Validation Loss")
#axs[1].plot(train_acc_list_graph, label="Training Accuracy")
axs[1].plot(val_acc_list_graph, label="Validation Accuracy")
axs[1].legend()
axs[0].legend()
fig.canvas.draw()
fig.show()